## Chemical kinetics
We will start by looking at a classic example in chemical kinetics: [Robertson's example](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.53.8603).
It is a 3-species reaction system:
$$
A \overset{k_1}{\rightarrow} B \\
B + C \overset{k_2}{\rightarrow} A + C \\
2 B \overset{k_3}{\rightarrow} B + C
$$
where A, B and C represent three different chemical species (e.g. reactive molecules dissolved in water).

The rate of each process follows the law of mass action, i.e. the rate is proportional to the concentration of each reacting species (to the power of their multiplicity). The proportionality constant is known as the rate constant of the reaction ($k_1,\ k_2\ \&\ k_3$ in our case). If we denote the rate of each reaction:
$$
r_1 = k_1[A] \\
r_2 = k_2[B][C] \\
r_3 = k_3[B]^2
$$

$[A],\ [B],\ [C]$ denotes the concentration of respective species. We can now formulate a system of ordinary differential equations describing how the concentrations evolve over time:

$$
\frac{d[A]}{dt} = r_2 - r_1 \\
\frac{d[B]}{dt} = r_1 - r_2 - r_3 \\
\frac{d[C]}{dt} = r_3
$$

We will now express these differential equations symbolically using SymPy:

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
c = cA, cB, cC = sp.symbols('[A] [B] [C]')  # concentrations
k = k1, k2, k3 = sp.symbols('k_1 k_2 k_3')
r1, r2, r3 = k1*cA, k2*cB*cC, k3*cB**2
f = fA, fB, fC = r2 - r1, r1 - r2 - r3, r3
f

$\mathbf{f} \equiv \dot{\mathbf{y}}$ now represent our ODE system, where $\mathbf{y}$ is our state vector (concentrations). We will need a callback to evaluate $\mathbf{f}$ when we integrate this ODE system numerically (using ``scipy.integrate.odeint``), SymPy can provide us with this callback:

In [ ]:
f_cb = sp.lambdify(c+k, f)
f_cb(1, 2, 3, 4, 5, 6)  # [A] = 1, [B] = 2, [C] = 3, k1 = 4, k2 = 5, k3 = 6

In [ ]:
from itertools import chain
import numpy as np
from scipy.integrate import odeint

In [ ]:
def integrate(tend=3600, t0=1e-6, **kwargs):
    tout = np.logspace(np.log10(t0), np.log10(tend))  # 1 us to tend (if we assume SI units)
    k_vals = (0.04, 1e4, 3e7)  # rates from the litterature, note that k1 has a different dimensionality
    c0 = [1, 0, 0]  # we start with only A with unit concentration (number density)
    yout, info = odeint(lambda y, t, *args: f_cb(*chain(y, args)), c0, tout, args=k_vals, full_output=True, **kwargs)
    return tout, yout, info

In [ ]:
tout, yout, info = integrate(1e6, atol=1e-9, rtol=1e-9)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_result(tout, yout, info=None):
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    for i, label in enumerate('ABC'):
        for ax in axes:
            ax.plot(tout, yout[:, i], label=label)
    axes[1].set_xscale('log')
    axes[1].set_yscale('log')
    for ax in axes:
        ax.set_ylabel('$\mathrm{concentration / mol \cdot dm^{-3}}$')
        ax.set_xlabel('$\mathrm{time / s}$')
        ax.legend(loc='best')
    if info:
        print("The jacobian was evaluated %d times" % info['nje'][-1])

In [ ]:
plot_result(tout, yout, info)



If we look closer at the info-dictionary we will see that odeint (or rather LSODA which is the unerlying package) switched method from an explicit Adams method to an implicit Backward Differentiation Formula (BDF). It is common for chemical kinetics problems that the problem becomes stiff. The larger number of species, the bigger is the Jacobian matrix.

By default the solver will approximate the elements in the Jacobian matrix by taking finite differences of $\mathbf{f}$. This is often works quite satisfactorlily but for larger systems it sometimes fails. A more robust (and faster) approach is to provide a callback which evaluates an analytic Jacobian. Using SymPy we can do this quite effortlessly:

In [ ]:
J = sp.Matrix(len(c), 1, f).jacobian(c)
J

In [ ]:
J_cb = sp.lambdify(c+k, J)
J_cb(1, 2, 3, 4, 5, 6)

In [ ]:
tout, yout, info = integrate(1e6, Dfun=lambda y, t, *args: J_cb(*chain(y, args)))

In [ ]:
plot_result(tout, yout, info)

We see that the solver needed to evaluate the Jacobian fewer times (due to it being essentially exact this time around). For larger systems the impact of an analytic Jacobian is often even greater (being the difference between a failed and succuessul integration).